In [1]:
# !pip install plotly
# !pip install pythainlp
# !pip install stop_words
# !pip install nltk

### Import Library

In [2]:
# import libraries
import pandas as pd
import plotly.express as px
import re
import nltk
nltk.download('words')
nltk.download('omw-1.4')

[nltk_data] Downloading package words to C:\Users\FACT-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\FACT-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
import os, sys

sys.path.insert(0,os.path.realpath(os.path.join(os.path.pardir, '..')))

from config import cfg
from utils.utils import *

[nltk_data] Downloading package words to C:\Users\FACT-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


### Import CSV

In [4]:
song_df = pd.read_csv(os.path.join(cfg.DATA.BASE_PATH, "35_song_per_artist.csv"))
song_df.dtypes

artist       object
song_name    object
href         object
lyrics       object
lines         int64
dtype: object

## Data Preparation

### Clean Data

In [5]:
# remove !@#$%^&*()_+

song_df['lyrics'] = clean(song_df['lyrics'], special_marks=cfg.SPECIAL_MARK, verbose=0)

# Remove '\n'
song_df['lyrics'] = song_df['lyrics'].map(lambda s: re.sub('\n', '', s))

100%|██████████| 24/24 [00:00<00:00, 132.21it/s]


In [6]:
# count number of lines
song_df['lines'] = song_df['lyrics'].map(lambda t: len(re.findall(r'\r\n', t)))

### Tokenise

In [7]:
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus.common import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
# from pythainlp.ulmfit import process_thai

In [8]:
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))
p_stemmer = PorterStemmer()

In [9]:
song_df['lyrics']

0       ไม่ว่าจะเป็นยังไง Baby its Okay\rไม่ว่าจะเกิดอ...
1       จะยอมนั่งจรวด ไปตรวจดาวอังคาร\rหากว่าที่แห่งนั...
2       ถ้าเคยพบเจอ กำแพงที่ดูทั้งใหญ่และสูงชัน\rเธอรู...
3       อยู่ตรงนี้แค่เพียงลำพัง\rกับความเหงาที่เป็นดั่...
4       แค่ตัวคนเดียวไม่ตายล่ะมั้ง\rถามใจกี่ครั้งก็ยัง...
                              ...                        
2235    พ่อตาผมโคตรโหด ผมไม่เคยเจอใครที่หนวดเยอะขนาดนี...
2236    ฉันไม่ได้ตั้งใจที่จะส่งไปหาเธอ\rหน้าจอมันเบลอเ...
2237    ทุกครั้งที่ฉันนะคิดจะลืม\rมันคือเวลาที่คิดถึงม...
2238    ไม่เป็นไรใช่ไหม\rนี่คือคำถาม หรือความห่วงใย\rถ...
2239    ไม่เคยมีใครน่ารักเท่ากับเธอ\rเจอกี่ครั้งก็ยังล...
Name: lyrics, Length: 2240, dtype: object

In [10]:
# split words from each line

song_df['words'] = song_df['lyrics'].map(lambda s: re.sub(r'\r', '', s)).map(lambda text: split_word(text))

# This function may take longer than 2 minutes.

In [11]:
song_df[['lyrics','words']].tail()

,lyrics,words
2235,พ่อตาผมโคตรโหด ผมไม่เคยเจอใครที่หนวดเยอะขนาดนี...,"[พ่อตา, ผม, โคตร, โหด, ผม, เจอ, หนวด, ขนาด, พ่..."
2236,ฉันไม่ได้ตั้งใจที่จะส่งไปหาเธอ\rหน้าจอมันเบลอเ...,"[ตั้งใจ, ที่จะ, ไปหา, หน้าจอ, เบลอ, พิมพ์, พลา..."
2237,ทุกครั้งที่ฉันนะคิดจะลืม\rมันคือเวลาที่คิดถึงม...,"[ทุกครั้งที่, ลืม, เวลา, คิดถึง, ฝัน, แบบนี้, ..."
2238,ไม่เป็นไรใช่ไหม\rนี่คือคำถาม หรือความห่วงใย\rถ...,"[คำถาม, ความห่วงใย, แคร์, im, sorri, เลิก, สัก..."
2239,ไม่เคยมีใครน่ารักเท่ากับเธอ\rเจอกี่ครั้งก็ยังล...,"[น่ารัก, เจอ, กี่, ละเมอ, ยอมให้, รัก, เปล่า, ..."


In [12]:
# word count

song_df['n_words'] = song_df['words'].map(len)

In [13]:
song_df[['n_words','words']].head()

,n_words,words
0,174,"[babi, okay, ill, miss, day, กลับมา, come, bac..."
1,56,"[นั่ง, จรวด, ตรวจ, ดาวอังคาร, ว่าที่, ทิ้ง, ดา..."
2,117,"[เจอ, กำแพง, ดู, สูงชัน, รู้, คน, สร้าง, หลบ, ..."
3,115,"[ตรงนี้, เพียงลำพัง, ความเหงา, เงา, แต่ยังไง, ..."
4,153,"[ตัว, คนเดียว, ตาย, ล่ะ, มั้ง, ถาม, ใจ, กี่, ร..."


### EDA

In [14]:
# number of songs
print('number of songs: ', str(len(song_df)))

# number of artists
print('number of artists: ', str(len(song_df['artist'].unique())))

number of songs:  2240
number of artists:  64


In [15]:
# distribution songs per artist
song_count_df = song_df.groupby('artist')[['song_name']].count()

fig = px.histogram(song_count_df, x='song_name', title='Songs per artist', labels={'song_name': 'Songs'})
fig.show()

In [16]:
# distribution words per song
fig = px.histogram(song_df, x='n_words', title='Words per song')
fig.show()

Beware! This block can be run only once.

In [17]:
# add a column of 'words_str'
song_df['words_str'] = song_df['words'].map(lambda lst: ' '.join(lst))

In [18]:
song_df.head()

,artist,song_name,href,lyrics,lines,words,n_words,words_str
0,bird_thongchai,Okay,/music/thailyric/13588,ไม่ว่าจะเป็นยังไง Baby its Okay\rไม่ว่าจะเกิดอ...,0,"[babi, okay, ill, miss, day, กลับมา, come, bac...",174,babi okay ill miss day กลับมา come back babyil...
1,bird_thongchai,กว่าจักรวาล,/music/thailyric/13978,จะยอมนั่งจรวด ไปตรวจดาวอังคาร\rหากว่าที่แห่งนั...,0,"[นั่ง, จรวด, ตรวจ, ดาวอังคาร, ว่าที่, ทิ้ง, ดา...",56,นั่ง จรวด ตรวจ ดาวอังคาร ว่าที่ ทิ้ง ดาว ขอบ จ...
2,bird_thongchai,กำแพง,/music/thailyric/14111,ถ้าเคยพบเจอ กำแพงที่ดูทั้งใหญ่และสูงชัน\rเธอรู...,0,"[เจอ, กำแพง, ดู, สูงชัน, รู้, คน, สร้าง, หลบ, ...",117,เจอ กำแพง ดู สูงชัน รู้ คน สร้าง หลบ ใจ พัก ท้...
3,bird_thongchai,ชีวิตเดี่ยว,/music/thailyric/13796,อยู่ตรงนี้แค่เพียงลำพัง\rกับความเหงาที่เป็นดั่...,0,"[ตรงนี้, เพียงลำพัง, ความเหงา, เงา, แต่ยังไง, ...",115,ตรงนี้ เพียงลำพัง ความเหงา เงา แต่ยังไง ชิน สั...
4,bird_thongchai,ผู้ต้องหา,/music/thailyric/13825,แค่ตัวคนเดียวไม่ตายล่ะมั้ง\rถามใจกี่ครั้งก็ยัง...,0,"[ตัว, คนเดียว, ตาย, ล่ะ, มั้ง, ถาม, ใจ, กี่, ร...",153,ตัว คนเดียว ตาย ล่ะ มั้ง ถาม ใจ กี่ รีบ คน รู้...


## Export the clean data

In [19]:
song_df.to_pickle(os.path.join(cfg.DATA.BASE_PATH, "clean_th_songs_newlog.pkl"))